## P-set 4

Answer the following questions (in written form).

# **1) (10P)**

**In the annealing process the energy state of the quantum chip ...**
1. (❌) should remain in the minimum eigenstate (groundstate)
2. should transform from a higher energy state (excited state) to the groundstate.
3. should tranform from the groundstate to an excited state.

**Make a cross (x) next to the correct answer above.**

# **2) (30P)**

**What is the minimum of the following binary quadratic model E?** 
$$ E(x_1,x_2) = \begin{pmatrix}x_1 & x_2\end{pmatrix} \begin{pmatrix}1 & 2 \\ 0 & 1\end{pmatrix} \begin{pmatrix}x_1 \\ x_2\end{pmatrix} $$

On paper:

![calc](powSet2attache/photo_2023-11-16_23-46-21.jpg)

# **3) (30P)**

**You are given a binary quadratic problem to minimize $\min \vec{x}^TQ\vec{x}$, where $Q$ has values $\ge 0$. How can you change the values of the $Q$ matrix such that the minimum solution is NOT the trivial solution and why?**

Introduce a negative value in $Q$.
Das macht, dass eine oder mehrere der $\vec{x}$ inputs auch negative Ergäbnisse ergeben.

Das sieht dan zum Beispiel so aus:

$ Q = \begin{bmatrix} -a & b \\ c & -d \end{bmatrix} $

$ E(x_1,x_2) = -ax_1^2 + bx_1x_2 + cx_2x_1 - dx_2^2 $

On the offchance, dass da gemeint ist das die werte in $Q$ auch nach dem modifizieren $\ge 0$ sein müssen, dann geht das offensichtlich nicht, da wenn alle coeffizienten positiv immer die triviale lösung dasminimum enthält. Das gegenteil gilt natürlich auch: Wenn alle coeffizienten negativ sind ist das optimum imer bei der antitrivialen lösung.

# **4) (30P)**

**Think about Exercise 3 again: Give an equality constraint for all the variables (drinks), which makes sure, that you will not drink more than three drinks containing alcohol.**

## Recap:

In [ ]:
!pip install dwave-system dimod seaborn pandas
# import necessary modules
import matplotlib.pyplot as plt
import seaborn as sns


from dwave.samplers import SimulatedAnnealingSampler, SteepestDescentSolver # classical free optimizer (installed locally on your computer)
from dwave.system import DWaveSampler, EmbeddingComposite # commercial quantum optimizer (in the cloud)

from dimod import BQM # binary quadratic model object


import numpy as np 
import pandas as pd



In [4]:
# the menu is provided here as pandas.DataFrame
menu = pd.DataFrame({
    'Drink': ['Long Island Ice Tea', 'Mochito', 'Erdbeer Daiquiri', 'Heineken', 'Corona', 'Eristoff Ice', 'Hugo Spritzer', 'Cola', 'Mineralwasser', 'Leitungswasser'],
    'Euro': [5.4, 5.4, 5.4, 3.5, 4, 4, 3.5, 3, 2.5, 0],
    'Score': [70, 60, 50, 35, 35, 20, 25, -2, -5, -2]
})
menu

,Drink,Euro,Score
0,Long Island Ice Tea,5.4,70
1,Mochito,5.4,60
2,Erdbeer Daiquiri,5.4,50
3,Heineken,3.5,35
4,Corona,4.0,35
5,Eristoff Ice,4.0,20
6,Hugo Spritzer,3.5,25
7,Cola,3.0,-2
8,Mineralwasser,2.5,-5
9,Leitungswasser,0.0,-2


In [9]:
# define a binary quadratic model with BQM("BINARY") and use the menu to add the drinks as variables with the according price from the menu. (check the hint below)
bqm = BQM("BINARY")
for drink,euro in zip(menu.Drink, menu.Euro):
    bqm.add_variable(drink,euro)

#HINT: use `bqm.add_variable(variable,cost)`

In [10]:
# add a linear the drinking score constraint using bqm.add_linear_equality_constraint() (hint below)
bqm.add_linear_equality_constraint([(drink,score) for drink,score in zip(menu.Drink, menu.Score)],1,-100)

In [11]:
# set the optimization to the dwavve quantum computer (you will need a user account which provides an api token, come to me when you reached this section to get it!)
#optimizer = EmbeddingComposite(DWaveSampler(token='TOKEN')) 
optimizer = SteepestDescentSolver()

In [12]:
sampleset = optimizer.sample(bqm=bqm)
solution = sampleset.first.sample
solution 

{'Cola': 0,
 'Corona': 0,
 'Erdbeer Daiquiri': 0,
 'Eristoff Ice': 0,
 'Heineken': 1,
 'Hugo Spritzer': 0,
 'Leitungswasser': 0,
 'Long Island Ice Tea': 0,
 'Mineralwasser': 0,
 'Mochito': 1}

In [13]:
# get the drinks where the value in the dictionary is 1
drinks_with_value_1 = [drink for drink, value in solution.items() if value == 1]

# filter the DataFrame using isin
result = menu[menu.Drink.isin(drinks_with_value_1)]

In [14]:
# display your dinks of choice
result.Drink

1     Mochito
3    Heineken
Name: Drink, dtype: object

In [15]:
# get your spending and score
result.sum()[1:]

Euro     8.9
Score     95
dtype: object

## Antwort:

Erstmal müssen wir alle alkoholischen🍷🍷 drinks Markieren. Ich mach das im dataframe:

(Long Island Ice Tea, Mojito, Erdbeer Daiquiri, Heineken, Corona, Eristoff Ice, Hugo Spritzer)

In [18]:
menu = pd.DataFrame({
    'Drink': ['Long Island Ice Tea', 'Mochito', 'Erdbeer Daiquiri', 'Heineken', 'Corona', 'Eristoff Ice', 'Hugo Spritzer', 'Cola', 'Mineralwasser', 'Leitungswasser'],
    'Euro': [5.4, 5.4, 5.4, 3.5, 4, 4, 3.5, 3, 2.5, 0],
    'Score': [70, 60, 50, 35, 35, 20, 25, -2, -5, -2],
    'Alk': [True, True, True, True, True, True, True, False, False, False]
})
menu

,Drink,Euro,Score,Alk
0,Long Island Ice Tea,5.4,70,True
1,Mochito,5.4,60,True
2,Erdbeer Daiquiri,5.4,50,True
3,Heineken,3.5,35,True
4,Corona,4.0,35,True
5,Eristoff Ice,4.0,20,True
6,Hugo Spritzer,3.5,25,True
7,Cola,3.0,-2,False
8,Mineralwasser,2.5,-5,False
9,Leitungswasser,0.0,-2,False


Dann mach ich all das bis zu den constraints, weil da änder ich nichts:

In [20]:
bqm = BQM("BINARY")
for drink,euro in zip(menu.Drink, menu.Euro):
    bqm.add_variable(drink,euro)
    
bqm.add_linear_equality_constraint([(drink,score) for drink,score in zip(menu.Drink, menu.Score)],1,-100)

Ok. Jetzt mach ich nen neuen constraint, bei dem ich definiere:

$ x_{\text{Long Island Ice Tea}} + x_{\text{Mojito}} + x_{\text{Erdbeer Daiquiri}} + x_{\text{Heineken}} + x_{\text{Corona}} + x_{\text{Eristoff Ice}} + x_{\text{Hugo Spritzer}} = 3 $

In [22]:
bqm.add_linear_equality_constraint([(drink, int(alk)) for drink,alk in zip(menu.Drink, menu.Alk)], lagrange_multiplier=1, constant=3 )

Also probierens wir mal aus:

In [31]:
optimizer = SteepestDescentSolver()
sampleset = optimizer.sample(bqm=bqm)
solution = sampleset.first.sample
solution 

{'Cola': 1,
 'Corona': 0,
 'Erdbeer Daiquiri': 0,
 'Eristoff Ice': 1,
 'Heineken': 0,
 'Hugo Spritzer': 1,
 'Leitungswasser': 1,
 'Long Island Ice Tea': 0,
 'Mineralwasser': 0,
 'Mochito': 1}

In [32]:
drinks_with_value_1 = [drink for drink, value in solution.items() if value == 1]
result = menu[menu.Drink.isin(drinks_with_value_1)]

result.Drink

1           Mochito
5      Eristoff Ice
6     Hugo Spritzer
7              Cola
9    Leitungswasser
Name: Drink, dtype: object

In [33]:
result.sum()[1:]

Euro     15.9
Score     101
Alk         3
dtype: object

As you can see sind im res nur 3 Alkoholische dinger drinnen :)

![sucess](powSet2attache/great-success-yes.gif)